# Exploring ebay Car Sales Data

In this project a dataset scraped from ebay Kleinanzeigen (classifieds section of the German ebay website) is analyzed. The dataset was originally scraped and uploaded to Kaggle. This project focuses on cleaning a dataset and to apply the technics of *boolean indexing*, *method-chaining* and *aggregation*. The original dataset isn't available on Kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data).

At the start the libraries of numpy and pandas are imported:

In [1]:
import pandas as pd
import numpy as np

Then the dataset `autos.csv` is read into pandas:

In [2]:
autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

The dataset is inspected by plotting the five first and last entries of the autos dataset:

In [3]:
autos

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,0,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,0,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,0,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,0,35683,2016-04-05 16:45:07


A more detailed analysis can be performed by utilizing the `Dataframe.info()` method:

In [4]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

The dataset consists mostly of object types (strings) and a few integer types. Null values exist but are underrepresented in comparison to non-null values.
Regarding the column names it can be observed, that *camelcase* is used instead of the python-preferred *snakecase*. Also some entries are not defined clearly, so the column names are corrected as follows:

In [5]:
def clean_col(col):
    col = col.replace('yearOfRegistration', 'registration_year')
    col = col.replace('monthOfRegistration', 'registration_month')
    col = col.replace('notRepairedDamage', 'unrepaired_damage')
    col = col.replace('fuelType', 'fuel_type')
    col = col.replace('dateCreated', 'ad_created')
    col = col.replace('dateCrawled', 'date_crawled')
    col = col.replace('offerType', 'offer_type')
    col = col.replace('abtest', 'ab_test')
    col = col.replace('vehicleType', 'vehicle_type')
    col = col.replace('powerPS', 'power_PS')
    col = col.replace('nrOfPictures', 'nr_of_pictures')
    col = col.replace('postalCode', 'postal_code')
    col = col.replace('lastSeen', 'last_seen')
    return col
    
new_columns = []

for c in autos.columns:
    new_columns.append(clean_col(c))
autos.columns = new_columns

The dataset can be analyzed further with the `DataSet.describe()` method:

In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-10 15:36:24,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The column `price` is expected to contain numeric values. However no statistical data is provided.

In [7]:
autos['price'].head()

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

The column contains entries of the `object` (strings) type with the prefix **$** and commas for the indication of thousands. These attributes are removed in the following and the entries are converted to `integer`:

In [8]:
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['price'] = autos['price'].astype(int)
autos['price'].dtype

dtype('int64')

Since the information of the currency is stripped from the entries, it should be added to the column label to not lose information:

In [9]:
autos = autos.rename(columns={'price':'price_dollars'})

The same procedure is repeated for the `odometer` column:

In [10]:
autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['odometer'] = autos['odometer'].astype(int)
autos['odometer'].dtype
autos = autos.rename(columns={'odometer':'odometer_km'})

The column `nr_of_pictures` contains only `null` entries and can be neglected:

In [11]:
autos['nr_of_pictures'].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

In the following the data in `odometer_km` is further analyzed:

In [12]:
autos['odometer_km'].unique().shape

(13,)

The column contains 13 unique entries and be statistically summarized by:

In [13]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [14]:
autos['odometer_km'].value_counts().sort_index(ascending=False)


150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

Most cars belong to the category of > 150000 km.
Furthermore the same analysis is performed for the and `price_dollars`entries:

In [15]:
autos['price_dollars'].unique().shape

(2357,)

In [16]:
autos['price_dollars'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollars, dtype: float64

In [17]:
autos['price_dollars'].value_counts().head()

0       1421
500      781
1500     734
2500     643
1000     639
Name: price_dollars, dtype: int64

It can be observed that 1421 entries have a price of 0 dollars. These entries are not of interest and should be deleted:

In [18]:
autos = autos[autos['price_dollars'] > 0]

In [19]:
autos['price_dollars'].value_counts().head()

500     781
1500    734
2500    643
1000    639
1200    639
Name: price_dollars, dtype: int64

Now, the `date_crawled` and `last_seen` columns should be analyzed. The entries consist of `object` types containing the time stamps. Including only the day (`str.[:10]`), converting the distribution to percentage (`normalize=True`) and sorting the findings from earliest to latest (`sort_index(ascending=True`) the following instructions can be chained:

In [20]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025320
2016-03-06    0.014039
2016-03-07    0.036003
2016-03-08    0.033327
2016-03-09    0.033101
2016-03-10    0.032174
2016-03-11    0.032566
2016-03-12    0.036930
2016-03-13    0.015665
2016-03-14    0.036538
2016-03-15    0.034274
2016-03-16    0.029601
2016-03-17    0.031639
2016-03-18    0.012907
2016-03-19    0.034768
2016-03-20    0.037876
2016-03-21    0.037403
2016-03-22    0.033018
2016-03-23    0.032216
2016-03-24    0.029334
2016-03-25    0.031598
2016-03-26    0.032195
2016-03-27    0.031083
2016-03-28    0.034850
2016-03-29    0.034130
2016-03-30    0.033677
2016-03-31    0.031845
2016-04-01    0.033677
2016-04-02    0.035468
2016-04-03    0.038597
2016-04-04    0.036518
2016-04-05    0.013092
2016-04-06    0.003170
2016-04-07    0.001400
Name: date_crawled, dtype: float64

The crawling of data was performed quite evenly over the course of two months.

In [21]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)



2016-03-05    0.001070
2016-03-06    0.004323
2016-03-07    0.005393
2016-03-08    0.007431
2016-03-09    0.009613
2016-03-10    0.010663
2016-03-11    0.012372
2016-03-12    0.023796
2016-03-13    0.008893
2016-03-14    0.012598
2016-03-15    0.015871
2016-03-16    0.016447
2016-03-17    0.028078
2016-03-18    0.007349
2016-03-19    0.015830
2016-03-20    0.020647
2016-03-21    0.020647
2016-03-22    0.021367
2016-03-23    0.018527
2016-03-24    0.019762
2016-03-25    0.019206
2016-03-26    0.016797
2016-03-27    0.015645
2016-03-28    0.020873
2016-03-29    0.022355
2016-03-30    0.024764
2016-03-31    0.023796
2016-04-01    0.022788
2016-04-02    0.024928
2016-04-03    0.025196
2016-04-04    0.024476
2016-04-05    0.124766
2016-04-06    0.221824
2016-04-07    0.131909
Name: last_seen, dtype: float64

All data was last seen in the year 2016.
Now, the `registration_year` column is analyzed.

In [22]:
autos['registration_year'].describe()

count    48579.000000
mean      2004.753000
std         88.631663
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The mean registration year of the cars advertised on ebay Kleinanzeigen is the year 2004. However, the minimum and maximum numbers are not plausible. The earliest registration year is most likely later than 1900 and all registration data after 2016 is incorrect since the ads were seen last in 2016 (`last_seen` column). Consequently all rows outside the invertal of [1900, 2016] can be safely removed if enough data is left for further inspection. 
Consequently the number of entries outside this intervall is calculated first:

In [23]:
autos['registration_year'][~((autos['registration_year'] >= 1900) & (autos['registration_year'] <= 2016))].shape

(1886,)

There are 1886 entries outside this interval, which are neglectable in comparison to all data entries and are deleted in the following:

In [24]:
autos = autos[((autos['registration_year'] >= 1900) & (autos['registration_year'] <= 2016))]

In [25]:
autos['registration_year'].value_counts(normalize=True).sort_values(ascending=False).head(10)

2000    0.067590
2005    0.062879
1999    0.062086
2004    0.057889
2003    0.057803
2006    0.057203
2001    0.056475
2002    0.053241
1998    0.050607
2007    0.048765
Name: registration_year, dtype: float64

The data above shows the ten most prominent registration years in the data set.

Now the mean prices for cars of a specific brand should be analyzed. To limit the output, only the brands with  > 5% of the total market share (advertised cars) should be addressed. First all car brands with their number of advertised cars are printed:

In [26]:
autos['brand'].value_counts()

volkswagen        9865
bmw               5138
opel              5023
mercedes_benz     4504
audi              4041
ford              3265
renault           2201
peugeot           1393
fiat              1197
seat               853
skoda              766
nissan             713
mazda              709
smart              661
citroen            655
toyota             593
hyundai            468
sonstige_autos     461
volvo              427
mini               409
mitsubishi         384
honda              366
kia                330
alfa_romeo         310
porsche            286
suzuki             277
chevrolet          266
chrysler           164
dacia              123
daihatsu           117
jeep               106
subaru             100
land_rover          98
saab                77
jaguar              73
daewoo              70
trabant             65
rover               62
lancia              50
lada                27
Name: brand, dtype: int64

The brands with more than 5 % market share can be calculated by:

In [27]:
advertised_by_brand = autos['brand'].value_counts()
size = autos.shape[0] #first element of the tuple
selected_brands = advertised_by_brand[advertised_by_brand[:] > (0.05*size)]
print(selected_brands)

volkswagen       9865
bmw              5138
opel             5023
mercedes_benz    4504
audi             4041
ford             3265
Name: brand, dtype: int64


For these brands the calculation of the mean price of the advertised cars is performed:

In [28]:
#create empty dictionary to store the calculated data
mean_price_by_brand = {}

#access the label of selected_brands
selected_brands_labels = selected_brands.index

#loop through the dataset
for label in selected_brands_labels:
    selected_rows = autos[autos['brand'] == label]
    #calculate the average price for those selected rows
    mean_price = selected_rows['price_dollars'].mean()
    mean_price = round(int(mean_price))
    #create a new dictionary key with the calculated price as value
    mean_price_by_brand[label] = mean_price
    
print(mean_price_by_brand)    

{'volkswagen': 6729, 'bmw': 8571, 'opel': 5432, 'mercedes_benz': 30829, 'audi': 9336, 'ford': 7456}


It can be observed that Mercedes Benz has by far the most expensive advertised cars.

Furthermore the average mileage for those brands shall be calculated and compared to the average price. 

In [29]:
#create empty dictionary to store the calculated data
mean_mileage_by_brand = {}

#loop through the dataset
for label in selected_brands_labels:
    selected_rows = autos[autos['brand'] == label]
    #calculate the average mileage for those selected rows
    mean_mileage = selected_rows['odometer_km'].mean()
    #round and typecast to integer
    mean_mileage = int(round(mean_mileage))
    #create a new dictionary key with the calculated mileage as value
    mean_mileage_by_brand[label] = mean_mileage
    
print(mean_mileage_by_brand)

{'volkswagen': 128714, 'bmw': 132576, 'opel': 129314, 'mercedes_benz': 130793, 'audi': 129157, 'ford': 124243}


For better observability and comparison, both `dictionaries` are converted into `Series` and combined to a `DataFrame`:

In [30]:
#create series
mpbb_series = pd.Series(mean_price_by_brand)
mmbb_series = pd.Series(mean_mileage_by_brand)

#create dataframe
comparison_df = pd.DataFrame(mpbb_series, columns=['mean_price_dollar'])
#add column
comparison_df['mean_mileage_km'] = mmbb_series

comparison_df

,mean_price_dollar,mean_mileage_km
volkswagen,6729,128714
bmw,8571,132576
opel,5432,129314
mercedes_benz,30829,130793
audi,9336,129157
ford,7456,124243


It can be observed that the mean mileage per brand is uncorrelated with the mean price per brand as the mean mileage does not vary significantly between differentbrands.